In [1]:
import $file.common
import common._
import doobie._, doobie.implicits._
import cats._, cats.implicits._
import cats.effect.IO
import fs2.Stream

Compiling /Users/jserrano/Documents/tagless-final-tutorial/session1/common.sc

import $file.$     

import common._

import doobie._, doobie.implicits._

import cats._, cats.implicits._

import cats.effect.IO

import fs2.Stream

# Variation 5. DAO in Scala

This is not DAO-based code. No interfaces for now.

Common case classes for DAO interfaces.

In [ ]:
case class Country(code: String, name: String, capital: Option[Int])
case class City(id: Int, name: String, countryCode: String, population: Int)

### IO

In [ ]:
class CityQueriesIO(xa: Transactor[IO]){
    def getCityName(id: Int): IO[Option[String]] = 
        sql"select name from city where id = $id"
            .query[String].option.transact(xa)
    
    def getCityPopulation(id: Int): IO[Option[Int]] = 
        sql"select population from city where id = $id"
            .query[Int].option.transact(xa)
}

object CityQueriesIO extends CityQueriesIO(xa)

class CountryQueriesIO(xa: Transactor[IO]){
    def getAllCountries: IO[List[Country]] = 
        sql"select code, name, capital from country"
            .query[Country].to[List].transact(xa)
    
    def getCountryName(id: Int): IO[Option[String]] =
        sql"select name from country where id = $id"
            .query[String].option.transact(xa)
    
    def getCountryCapital(id: Int): IO[Option[Int]] =
        sql"select capital from country where id = $id"
            .query[Int].option.transact(xa)
}

object CountryQueriesIO extends CountryQueriesIO(xa)

No filter for IO, can't write our query easily. TBD.

In [ ]:
val largeCapitalsIO: IO[List[Option[(String, String)]]] = for {
    countries <- CountryQueriesIO.getAllCountries
    result <- countries.traverse{
        case Country(code,name,Some(capital)) => 
            CityQueriesIO.getCityName(capital).map(_.map((_, name)))
        case _ => None.pure[IO]
    }
} yield result

In [ ]:
largeCapitalsIO.unsafeRunSync.timed

BIG BIG BIG PROBLEM: no filters AND query avalance AND multiple connections

### `Stream[ConnectionIO, ?]`

In [ ]:
object CityQueries{
    def getCityName(id: Int): Stream[ConnectionIO, String] = 
        sql"select name from city where id = $id"
            .query[String].stream
    
    def getCityPopulation(id: Int): Stream[ConnectionIO,Int] = 
        sql"select population from city where id = $id"
            .query[Int].stream
}

object CountryQueries{
    def getAllCountries: Stream[ConnectionIO, Country] = 
        sql"select code, name, capital from country"
            .query[Country].stream
    
    def getCountryName(id: Int): Stream[ConnectionIO,String] =
        sql"select name from country where id = $id"
            .query[String].stream
    
    def getCountryCapital(id: Int): Stream[ConnectionIO, Int] =
        sql"select capital from country where id = $id"
            .query[Int].stream
}

import CityQueries._, CountryQueries._

In [ ]:
val largeCapitals: Stream[ConnectionIO,(String, String)] = for {
    Country(_, name, Some(capital)) <- getAllCountries
    population <- getCityPopulation(capital)
    if population > 8000000
    cityName <- getCityName(capital)
} yield (cityName, name)

In [ ]:
largeCapitals.compile
    .toList
    .transact(xa)
    .unsafeRunSync
    .timed

BIG PROBLEM: query avalanche (although single connection).